# Initial Testing with pretrained LLM (Gemma, Qwen)

## Install packages

In [25]:
%pip install python-dotenv huggingface_hub pandas transformers hf_xet accelerate
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu129

Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu129
Note: you may need to restart the kernel to use updated packages.


## Import packages

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from huggingface_hub import login
from huggingface_hub import InferenceClient
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import gc

## Login to HuggingFace

In [2]:
load_dotenv()
hf_token = os.getenv("HF_TOKEN")
login(token=hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


## Load Dataset (kaggle)

In [3]:
df = pd.read_csv("./reviews.csv")
reviews = df["text"].dropna().tolist()
print(df.head())
print(len(df))
print(len(reviews))

                     business_name    author_name  \
0  Haci'nin Yeri - Yigit Lokantasi    Gulsum Akar   
1  Haci'nin Yeri - Yigit Lokantasi  Oguzhan Cetin   
2  Haci'nin Yeri - Yigit Lokantasi     Yasin Kuyu   
3  Haci'nin Yeri - Yigit Lokantasi     Orhan Kapu   
4  Haci'nin Yeri - Yigit Lokantasi     Ozgur Sati   

                                                text  \
0  We went to Marmaris with my wife for a holiday...   
1  During my holiday in Marmaris we ate here to f...   
2  Prices are very affordable. The menu in the ph...   
3  Turkey's cheapest artisan restaurant and its f...   
4  I don't know what you will look for in terms o...   

                                               photo  rating  \
0         dataset/taste/hacinin_yeri_gulsum_akar.png       5   
1        dataset/menu/hacinin_yeri_oguzhan_cetin.png       4   
2  dataset/outdoor_atmosphere/hacinin_yeri_yasin_...       3   
3  dataset/indoor_atmosphere/hacinin_yeri_orhan_k...       5   
4           dataset/menu

## Prompt design

In [4]:
labels = ["Advertisement", "Irrelevant Content", "Rant Without Visit", "Clean"]
prompts = [f"Classify the following Google review into one of these categories and output only the label: {labels}.\nReview: \"{text}\"" for text in reviews]
print(prompts[0])

Classify the following Google review into one of these categories and output only the label: ['Advertisement', 'Irrelevant Content', 'Rant Without Visit', 'Clean'].
Review: "We went to Marmaris with my wife for a holiday. We chose this restaurant as a place for dinner based on the reviews and because we wanted juicy food. When we first went there was a serious queue. You proceed by taking the food you want in the form of an open buffet. Both vegetable dishes and meat dishes were plentiful. There was also dessert for those who wanted it. After you get what you want you pay at the cashier. They don't go through cards they work in cash. There was a lot of food variety. And the food prices were unbelievably cheap. We paid only 84 TL for all the meals here. It included buttermilk and bread. But unfortunately I can't say it's too clean as a place.."


## Inference pipeline

### Cloud

In [6]:
client = InferenceClient()
result = client.chat_completion(
    model="Qwen/Qwen3-8B",
    messages=[
        {"role": "user", "content": prompts[0]}
    ]
)
print(result.choices[0].message.content)

KeyboardInterrupt: 

### Local

#### Clear CUDA memory

In [5]:
torch.cuda.empty_cache()
gc.collect()

124

#### Load LLM (Qwen)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-8B")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-8B",
    torch_dtype="auto",
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


NameError: name 'index' is not defined

#### Inference

In [16]:
messages = [
    {"role": "user", "content": prompts[0]}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
print(prompts[0])
print(tokenizer.decode(output_ids[0:], skip_special_tokens=True).strip("\n"))

Classify the following Google review into one of these categories and output only the label: ['Advertisement', 'Irrelevant Content', 'Rant Without Visit', 'Clean'].
Review: "We went to Marmaris with my wife for a holiday. We chose this restaurant as a place for dinner based on the reviews and because we wanted juicy food. When we first went there was a serious queue. You proceed by taking the food you want in the form of an open buffet. Both vegetable dishes and meat dishes were plentiful. There was also dessert for those who wanted it. After you get what you want you pay at the cashier. They don't go through cards they work in cash. There was a lot of food variety. And the food prices were unbelievably cheap. We paid only 84 TL for all the meals here. It included buttermilk and bread. But unfortunately I can't say it's too clean as a place.."
Clean
